# Segunda entrega - 51940 Cueva Tomas
En este notebook se desarrolla un proceso ETL. El mismo consiste de la extracción de datos de la API de CoinGecko, su transformación y filtrado a través de la biblioteca Pandas, finalizando con la carga de los datos utilizando PySpark.

## Instalo bibliotecas y llamo a los modulos del paquete etl de este directorio

In [1]:
!pip install numpy pandas requests SQLAlchemy psycopg2-binary pyspark

In [2]:
from etl import(
    extract_CoinGecko_API as extract,
    transform_CoinGecko_API as transform,
    load_pyspark_redshift_connectr as load
)
import pandas as pd
import numpy as np

## Inicializo las clases de los modulo.

In [3]:
extr = extract.Extract("https://api.coingecko.com/api/v3/")
ld = load.Load()

## 100 criptos con mayor capitalización bursatil
En las siguientes celdas se extraen los datos de las 100 criptomonedas con mayor capitalizazción bursatil, se convierten en un dataframe y se cargan a su tabla correspondiente de redshift,

In [4]:
json_top = extr.get_criptos_top()

In [5]:
df_top = transform.transformation_top(json_top)

In [6]:
df_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                100 non-null    object 
 1   symbol                            100 non-null    object 
 2   name                              100 non-null    object 
 3   current_price                     100 non-null    float64
 4   market_cap                        100 non-null    int64  
 5   market_cap_rank                   100 non-null    int64  
 6   total_volume                      100 non-null    float64
 7   high_24h                          100 non-null    float64
 8   low_24h                           100 non-null    float64
 9   price_change_24h                  100 non-null    float64
 10  price_change_percentage_24h       100 non-null    float64
 11  market_cap_change_24h             100 non-null    float64
 12  market_ca

In [6]:
table = "criptos_market_cap"
ld.execute(df_top, table)

### Market chart de las top 10
Utilizando los datos del dataframe del top 100, vamos a obtener el market chart de las 10 primeras del top.

In [7]:
id_column = df_top['id']

In [8]:
id_array = id_column.to_numpy()

Luego ded obtener el array con todos los id, vamos a iterar 10 veces sobre el contenido del array, vamos a extraer los datos y transformarlos con las funciones de los modulos "extr" y "transform", para finalmente concatenarlos en un unico DataFrame.

In [9]:
df_all_market_charts = pd.DataFrame(columns = ['timestamp', 'prices', 'market_caps', 'total_volumes', 'cripto'])

for i in range(0,10):
    data = extr.get_market_chart(id_array[i])
    df = transform.json_to_df_market_chart(data,id_array[i])
    df_all_market_charts = pd.concat([df_all_market_charts, df], ignore_index=True)

Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa


In [10]:
df_all_market_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25613 entries, 0 to 25612
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp      25613 non-null  object 
 1   prices         25613 non-null  float64
 2   market_caps    25610 non-null  float64
 3   total_volumes  25613 non-null  float64
 4   cripto         25613 non-null  object 
dtypes: float64(3), object(2)
memory usage: 1000.6+ KB


In [11]:
table = "market_chart_criptos"
ld.execute(df_all_market_charts, table)

Ejecutando ETL
Convertir el DataFrame de pandas a un PySpark DataFrame
DataFrame[timestamp: bigint, prices: double, market_caps: double, total_volumes: double, cripto: string]
Cargar el PySpark DataFrame en Redshift
Dataframe subido
